# A-LSA Demonstration Notebook

**Adaptive Latent Semantic Analysis for Binary Text Classification**

Author: Isaac Touza  
Institution: Université de Maroua, Cameroun  
Date: January 2026

---

This notebook demonstrates the complete A-LSA workflow across **three diverse datasets**:

1. **20 Newsgroups** - Topic classification (comp.graphics vs rec.sport.hockey)
2. **IMDb Movie Reviews** - Sentiment analysis (positive vs negative)
3. **SMS Spam Collection** - Spam detection (spam vs ham)

For each dataset, we will:
- Load and preprocess the data
- Train A-LSA and baseline models
- Evaluate and compare performance
- Visualize results and characteristic terms

## 1. Setup and Imports

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path
sys.path.append('..')

from src.alsa import AdaptiveLSA
from src.baselines import get_baseline_models, create_imbalanced_dataset
from src.evaluation import evaluate_model, compare_models
from src.visualization import (
    plot_tsne_visualization,
    plot_characteristic_terms,
    plot_performance_comparison,
    plot_sensitivity_to_k,
    plot_imbalance_impact
)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Configuration
RANDOM_STATE = 42
TEST_SIZE = 0.2
N_COMPONENTS = 100

print("✓ Imports successful")

## 2. Dataset Loading Functions

In [ ]:
def load_20newsgroups_data():
    """Load 20 Newsgroups dataset."""
    categories = ('comp.graphics', 'rec.sport.hockey')
    
    data = fetch_20newsgroups(
        subset='all',
        categories=categories,
        remove=('headers', 'footers', 'quotes'),
        shuffle=True,
        random_state=RANDOM_STATE
    )
    
    return data.data, data.target, categories


def load_imdb_data(data_dir='../data/imdb', max_samples=10000):
    """Load IMDb Movie Reviews dataset."""
    texts = []
    labels = []
    
    if not os.path.exists(os.path.join(data_dir, 'train', 'pos')):
        print(f"⚠ IMDb dataset not found at {data_dir}")
        print("  Download from: https://ai.stanford.edu/~amaas/data/sentiment/")
        return None, None, None
    
    # Load data
    for split in ['train', 'test']:
        for label_name, label_value in [('pos', 1), ('neg', 0)]:
            dir_path = os.path.join(data_dir, split, label_name)
            
            if not os.path.exists(dir_path):
                continue
            
            for filename in os.listdir(dir_path):
                if filename.endswith('.txt'):
                    with open(os.path.join(dir_path, filename), 'r', encoding='utf-8') as f:
                        texts.append(f.read())
                        labels.append(label_value)
    
    labels = np.array(labels)
    
    # Subsample for efficiency
    if len(texts) > max_samples:
        indices = np.random.RandomState(RANDOM_STATE).choice(
            len(texts), max_samples, replace=False
        )
        texts = [texts[i] for i in indices]
        labels = labels[indices]
    
    return texts, labels, ('Negative', 'Positive')


def load_sms_spam_data(data_path='../data/sms_spam/SMSSpamCollection'):
    """Load SMS Spam Collection dataset."""
    if not os.path.exists(data_path):
        print(f"⚠ SMS Spam dataset not found at {data_path}")
        print("  Download from: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection")
        return None, None, None
    
    texts = []
    labels = []
    
    with open(data_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t', 1)
            if len(parts) == 2:
                label, text = parts
                texts.append(text)
                labels.append(1 if label == 'spam' else 0)
    
    labels = np.array(labels)
    
    return texts, labels, ('Ham', 'Spam')


print("✓ Dataset loading functions defined")

---

# Part 1: 20 Newsgroups Dataset

Topic classification between comp.graphics and rec.sport.hockey

---

## 3.1 Load 20 Newsgroups

In [ ]:
print("Loading 20 Newsgroups dataset...")
X_ng, y_ng, categories_ng = load_20newsgroups_data()

print(f"✓ Loaded {len(X_ng)} documents")
print(f"  - {categories_ng[0]}: {np.sum(y_ng == 0)} documents ({100*np.mean(y_ng == 0):.1f}%)")
print(f"  - {categories_ng[1]}: {np.sum(y_ng == 1)} documents ({100*np.mean(y_ng == 1):.1f}%)")

# Split data
X_ng_train, X_ng_test, y_ng_train, y_ng_test = train_test_split(
    X_ng, y_ng, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_ng
)

print(f"\nTraining: {len(X_ng_train)} | Test: {len(X_ng_test)}")

## 3.2 Train Models on 20 Newsgroups

In [ ]:
# Initialize models with optimized A-LSA parameters
print("Initializing models...")
models_ng = {}
models_ng['A-LSA'] = AdaptiveLSA(
    n_components=N_COMPONENTS,
    normalize_energies=True,
    optimize_threshold=True,
    random_state=RANDOM_STATE
)
models_ng.update(get_baseline_models(n_components=N_COMPONENTS, random_state=RANDOM_STATE))

# Train and evaluate
print("\nTraining and evaluating models...")
results_ng = compare_models(
    models=models_ng,
    X_train=X_ng_train,
    y_train=y_ng_train,
    X_test=X_ng_test,
    y_test=y_ng_test,
    n_cv_folds=5,
    random_state=RANDOM_STATE
)

results_ng['Dataset'] = '20 Newsgroups'

print("\n" + "="*80)
print("20 NEWSGROUPS RESULTS")
print("="*80)
print(results_ng[['Model', 'Test F1 (macro)', 'Test Accuracy']].to_string(index=False))

## 3.3 Visualize 20 Newsgroups Results

In [ ]:
# Characteristic terms
alsa_ng = models_ng['A-LSA']
char_terms_ng = alsa_ng.get_characteristic_terms(n_terms=10)

plot_characteristic_terms(
    terms_pos=char_terms_ng['positive'],
    terms_neg=char_terms_ng['negative'],
    class_names=categories_ng,
    n_terms=10
)

print(f"\nTop {categories_ng[1]} terms:", [t for t, _ in char_terms_ng['positive'][:5]])
print(f"Top {categories_ng[0]} terms:", [t for t, _ in char_terms_ng['negative'][:5]])

In [ ]:
# t-SNE visualization
z_pos_ng, z_neg_ng = alsa_ng.get_latent_projections(X_ng_test)

plot_tsne_visualization(
    z_pos=z_pos_ng,
    z_neg=z_neg_ng,
    y_true=y_ng_test,
    class_names=categories_ng,
    random_state=RANDOM_STATE
)

---

# Part 2: IMDb Movie Reviews

Sentiment analysis (positive vs negative reviews)

---

## 4.1 Load IMDb Dataset

In [ ]:
print("Loading IMDb dataset...")
X_imdb, y_imdb, categories_imdb = load_imdb_data()

if X_imdb is not None:
    print(f"✓ Loaded {len(X_imdb)} reviews")
    print(f"  - {categories_imdb[0]}: {np.sum(y_imdb == 0)} reviews ({100*np.mean(y_imdb == 0):.1f}%)")
    print(f"  - {categories_imdb[1]}: {np.sum(y_imdb == 1)} reviews ({100*np.mean(y_imdb == 1):.1f}%)")
    
    # Split data
    X_imdb_train, X_imdb_test, y_imdb_train, y_imdb_test = train_test_split(
        X_imdb, y_imdb, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_imdb
    )
    
    print(f"\nTraining: {len(X_imdb_train)} | Test: {len(X_imdb_test)}")
else:
    print("⚠ Skipping IMDb dataset (not available)")

## 4.2 Train Models on IMDb

In [ ]:
if X_imdb is not None:
    # Initialize models with optimized A-LSA parameters
    print("Initializing models...")
    models_imdb = {}
    models_imdb['A-LSA'] = AdaptiveLSA(
        n_components=N_COMPONENTS,
        normalize_energies=True,
        optimize_threshold=True,
        random_state=RANDOM_STATE
    )
    models_imdb.update(get_baseline_models(n_components=N_COMPONENTS, random_state=RANDOM_STATE))
    
    # Train and evaluate
    print("\nTraining and evaluating models...")
    results_imdb = compare_models(
        models=models_imdb,
        X_train=X_imdb_train,
        y_train=y_imdb_train,
        X_test=X_imdb_test,
        y_test=y_imdb_test,
        n_cv_folds=5,
        random_state=RANDOM_STATE
    )
    
    results_imdb['Dataset'] = 'IMDb'
    
    print("\n" + "="*80)
    print("IMDB RESULTS")
    print("="*80)
    print(results_imdb[['Model', 'Test F1 (macro)', 'Test Accuracy']].to_string(index=False))

## 4.3 Visualize IMDb Results

In [ ]:
if X_imdb is not None:
    # Characteristic terms
    alsa_imdb = models_imdb['A-LSA']
    char_terms_imdb = alsa_imdb.get_characteristic_terms(n_terms=10)
    
    plot_characteristic_terms(
        terms_pos=char_terms_imdb['positive'],
        terms_neg=char_terms_imdb['negative'],
        class_names=categories_imdb,
        n_terms=10
    )
    
    print(f"\nTop {categories_imdb[1]} sentiment terms:", [t for t, _ in char_terms_imdb['positive'][:5]])
    print(f"Top {categories_imdb[0]} sentiment terms:", [t for t, _ in char_terms_imdb['negative'][:5]])

---

# Part 3: SMS Spam Collection

Spam detection (spam vs ham messages)

---

## 5.1 Load SMS Spam Dataset

In [ ]:
print("Loading SMS Spam dataset...")
X_sms, y_sms, categories_sms = load_sms_spam_data()

if X_sms is not None:
    print(f"✓ Loaded {len(X_sms)} messages")
    print(f"  - {categories_sms[0]}: {np.sum(y_sms == 0)} messages ({100*np.mean(y_sms == 0):.1f}%)")
    print(f"  - {categories_sms[1]}: {np.sum(y_sms == 1)} messages ({100*np.mean(y_sms == 1):.1f}%)")
    
    # Split data
    X_sms_train, X_sms_test, y_sms_train, y_sms_test = train_test_split(
        X_sms, y_sms, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_sms
    )
    
    print(f"\nTraining: {len(X_sms_train)} | Test: {len(X_sms_test)}")
else:
    print("⚠ Skipping SMS Spam dataset (not available)")

## 5.2 Train Models on SMS Spam

In [ ]:
if X_sms is not None:
    # Initialize models with optimized A-LSA parameters for SMS (short texts)
    print("Initializing models...")
    models_sms = {}
    models_sms['A-LSA'] = AdaptiveLSA(
        n_components=75,  # Optimal for SMS Spam
        min_df=1,  # Lower min_df for short texts
        normalize_energies=True,
        optimize_threshold=True,
        random_state=RANDOM_STATE
    )
    models_sms.update(get_baseline_models(n_components=N_COMPONENTS, random_state=RANDOM_STATE))
    
    # Train and evaluate
    print("\nTraining and evaluating models...")
    results_sms = compare_models(
        models=models_sms,
        X_train=X_sms_train,
        y_train=y_sms_train,
        X_test=X_sms_test,
        y_test=y_sms_test,
        n_cv_folds=5,
        random_state=RANDOM_STATE
    )
    
    results_sms['Dataset'] = 'SMS Spam'
    
    print("\n" + "="*80)
    print("SMS SPAM RESULTS")
    print("="*80)
    print(results_sms[['Model', 'Test F1 (macro)', 'Test Accuracy']].to_string(index=False))

## 5.3 Visualize SMS Spam Results

In [ ]:
if X_sms is not None:
    # Characteristic terms
    alsa_sms = models_sms['A-LSA']
    char_terms_sms = alsa_sms.get_characteristic_terms(n_terms=10)
    
    plot_characteristic_terms(
        terms_pos=char_terms_sms['positive'],
        terms_neg=char_terms_sms['negative'],
        class_names=categories_sms,
        n_terms=10
    )
    
    print(f"\nTop {categories_sms[1]} terms:", [t for t, _ in char_terms_sms['positive'][:5]])
    print(f"Top {categories_sms[0]} terms:", [t for t, _ in char_terms_sms['negative'][:5]])

---

# Part 4: Cross-Dataset Comparison

Compare A-LSA performance across all three datasets

---

## 6. Compare Results Across Datasets

In [ ]:
# Combine all results
all_results = [results_ng]

if X_imdb is not None:
    all_results.append(results_imdb)
    
if X_sms is not None:
    all_results.append(results_sms)

combined_results = pd.concat(all_results, ignore_index=True)

print("\n" + "="*80)
print("CROSS-DATASET COMPARISON")
print("="*80)
print(combined_results[['Dataset', 'Model', 'Test F1 (macro)', 'Test Accuracy']].to_string(index=False))

In [ ]:
# Visualize A-LSA performance across datasets
alsa_results = combined_results[combined_results['Model'] == 'A-LSA'].copy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# F1 scores
axes[0].bar(alsa_results['Dataset'], alsa_results['Test F1 (macro)'], color='steelblue', alpha=0.7)
axes[0].set_ylabel('F1 Score (macro)', fontweight='bold')
axes[0].set_title('A-LSA F1 Performance Across Datasets', fontweight='bold')
axes[0].set_ylim([0.7, 1.0])
axes[0].grid(True, alpha=0.3, axis='y')

# Accuracy
axes[1].bar(alsa_results['Dataset'], alsa_results['Test Accuracy'], color='coral', alpha=0.7)
axes[1].set_ylabel('Accuracy', fontweight='bold')
axes[1].set_title('A-LSA Accuracy Across Datasets', fontweight='bold')
axes[1].set_ylim([0.7, 1.0])
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated A-LSA across three diverse datasets:

### 20 Newsgroups
- ✓ Topic classification task
- ✓ Balanced dataset
- ✓ Medium-length documents

### IMDb Movie Reviews
- ✓ Sentiment analysis task
- ✓ Balanced dataset
- ✓ Long documents (~230 words)

### SMS Spam Collection
- ✓ Spam detection task
- ✓ Imbalanced dataset (13% spam)
- ✓ Very short texts (~80 characters)
- ✓ **Optimal performance with k=75, min_df=1**

**Key Findings:**
- A-LSA achieves competitive or superior performance across all datasets
- Dual latent spaces effectively capture class-specific semantics
- Method is robust to varying text lengths and class imbalance
- Characteristic terms provide interpretable insights
- **Energy normalization and threshold optimization are critical for imbalanced data**
- **Performance is relatively stable across k values (50-150)**

**Recent Improvements:**
- ✓ Energy normalization by explained variance ratio
- ✓ Validation-based threshold optimization
- ✓ Adaptive parameters for different text lengths
- ✓ Robust to class imbalance (1:10 ratio)

In [ ]:
# Test robustness to imbalance on 20 Newsgroups dataset
print("Testing robustness to class imbalance on 20 Newsgroups...")

imbalance_ratios = [1.0, 0.5, 0.33, 0.2, 0.1]  # 1:1 to 1:10
f1_scores_imb = {
    'A-LSA': [],
    'A-LSA (no θ adjustment)': [],
    'Logistic Regression': []
}

for ratio in imbalance_ratios:
    print(f"\nTesting ratio {ratio:.2f} (1:{int(1/ratio)})...")
    
    # Create imbalanced training set
    if ratio < 1.0:
        X_train_imb, y_train_imb = create_imbalanced_dataset(
            X_ng_train,
            y_ng_train,
            ratio=ratio,
            random_state=RANDOM_STATE
        )
    else:
        X_train_imb = X_ng_train
        y_train_imb = y_ng_train
    
    print(f"  Training set: {len(X_train_imb)} samples")
    print(f"  Positive: {np.sum(y_train_imb == 1)}, Negative: {np.sum(y_train_imb == 0)}")
    
    # A-LSA with threshold optimization
    alsa_opt = AdaptiveLSA(
        n_components=N_COMPONENTS,
        normalize_energies=True,
        optimize_threshold=True,
        random_state=RANDOM_STATE
    )
    alsa_opt.fit(X_train_imb, y_train_imb)
    alsa_opt_metrics = evaluate_model(alsa_opt, X_ng_test, y_ng_test)
    f1_scores_imb['A-LSA'].append(alsa_opt_metrics['F1-score (macro)'])
    print(f"  A-LSA F1: {alsa_opt_metrics['F1-score (macro)']:.4f} (θ={alsa_opt.theta_:.4f})")
    
    # A-LSA without threshold optimization
    alsa_no_opt = AdaptiveLSA(
        n_components=N_COMPONENTS,
        normalize_energies=True,
        optimize_threshold=False,
        random_state=RANDOM_STATE
    )
    alsa_no_opt.fit(X_train_imb, y_train_imb)
    alsa_no_opt.theta_ = 0.0  # Force θ = 0
    alsa_no_opt_metrics = evaluate_model(alsa_no_opt, X_ng_test, y_ng_test)
    f1_scores_imb['A-LSA (no θ adjustment)'].append(alsa_no_opt_metrics['F1-score (macro)'])
    print(f"  A-LSA (no θ) F1: {alsa_no_opt_metrics['F1-score (macro)']:.4f}")
    
    # Logistic Regression baseline
    lr_models = get_baseline_models(random_state=RANDOM_STATE)
    lr = lr_models['Logistic Regression']
    lr.fit(X_train_imb, y_train_imb)
    lr_metrics = evaluate_model(lr, X_ng_test, y_ng_test)
    f1_scores_imb['Logistic Regression'].append(lr_metrics['F1-score (macro)'])
    print(f"  LR F1: {lr_metrics['F1-score (macro)']:.4f}")

# Plot results
plot_imbalance_impact(
    imbalance_ratios=imbalance_ratios,
    f1_scores=f1_scores_imb
)

print("\n✓ Imbalance analysis complete")

## 7.2 Robustness to Class Imbalance

How well does A-LSA handle varying degrees of class imbalance?

In [ ]:
# Test sensitivity to k on 20 Newsgroups dataset
print("Testing sensitivity to k on 20 Newsgroups...")

k_values = [10, 25, 50, 75, 100, 125, 150, 200]
f1_scores_k = {
    'A-LSA': [],
    'LSA + Logistic Regression': []
}

for k in k_values:
    print(f"\nTesting k={k}...")
    
    # A-LSA
    alsa_k = AdaptiveLSA(
        n_components=k,
        normalize_energies=True,
        optimize_threshold=True,
        random_state=RANDOM_STATE
    )
    alsa_k.fit(X_ng_train, y_ng_train)
    alsa_metrics = evaluate_model(alsa_k, X_ng_test, y_ng_test)
    f1_scores_k['A-LSA'].append(alsa_metrics['F1-score (macro)'])
    print(f"  A-LSA F1: {alsa_metrics['F1-score (macro)']:.4f}")
    
    # LSA + LR
    lsa_lr_models = get_baseline_models(n_components=k, random_state=RANDOM_STATE)
    lsa_lr = lsa_lr_models['LSA + Logistic Regression']
    lsa_lr.fit(X_ng_train, y_ng_train)
    lsa_lr_metrics = evaluate_model(lsa_lr, X_ng_test, y_ng_test)
    f1_scores_k['LSA + Logistic Regression'].append(lsa_lr_metrics['F1-score (macro)'])
    print(f"  LSA+LR F1: {lsa_lr_metrics['F1-score (macro)']:.4f}")

# Plot results
plot_sensitivity_to_k(
    k_values=k_values,
    f1_scores=f1_scores_k,
    optimal_k=100
)

print("\n✓ Sensitivity analysis complete")

## 7.1 Sensitivity to Latent Dimension k

How does performance vary with the dimensionality of the latent space?

---

# Part 5: Sensitivity Analysis

Analyze A-LSA robustness to hyperparameters and data conditions

---